<center>
<h1 id=""><img src="https://storage.googleapis.com/unskript-website/assets/favicon.png" alt="unSkript.com" width="100" height="100"></h1>
<h1 id="unSkript-Runbooks">unSkript Runbooks</h1>
<div class="alert alert-block alert-success">
<h3 id="Objective"><strong>Objective</strong></h3>
<strong>Detach EC2 Instance from Auto Scaling Group</strong></div>
</center><center>
<h2 id="Detach-EC2-Instance-from-Auto-Scaling-Group"><strong>Detach EC2 Instance from Auto Scaling Group</strong></h2>
</center>
<h1 id="Steps-Overview"><sub>Steps Overview</sub></h1>
<p>1. &nbsp;Get Unhealthy instances from ASG</p>
<p>2.&nbsp; AWS Detach Instances From AutoScaling Group</p>

In [11]:
if instance_id and not region:
    raise SystemExit("Provide region for the instance!")

<h3 id="Get-AWS-AutoScaling-Group-Instances">Get AWS AutoScaling Group Instances</h3>
<p>Using unSkript's <strong>Get AWS AutoScaling Group Instances</strong> action we list all the EC2 instances for a given region with Auto Scaling Group name. This action only executes if the instance_id and region have been given as parameters.</p>
<ul>
<li><strong>Input parameters:</strong>&nbsp; <code>instance_ids, region</code></li>
<li><strong>Output variable:</strong>&nbsp; <code>asg_instance</code></li>
</ul>

In [3]:
##  Copyright (c) 2021 unSkript, Inc
##  All rights reserved.
##
from typing import List, Dict
from pydantic import BaseModel, Field
from tabulate import tabulate
from botocore.exceptions import ClientError
import pprint

from beartype import beartype
@beartype
def aws_get_auto_scaling_instances_printer(output):
    if output is None:
        return
    print(tabulate(output, headers='keys'))


@beartype
def aws_get_auto_scaling_instances(handle, instance_ids: list, region: str) -> List:
    """aws_get_auto_scaling_instances List of Dict with instanceId and attached groups.

        :type handle: object
        :param handle: Object returned from task.validate(...).

        :type instance_ids: list
        :param instance_ids: List of instances.

        :type region: string
        :param region: Region to filter instances.

        :rtype: List of Dict with instanceId and attached groups.
    """
    result = []
    ec2Client = handle.client('autoscaling', region_name=region)
    try:
        response = ec2Client.describe_auto_scaling_instances(InstanceIds=instance_ids)
        for group in response["AutoScalingInstances"]:
            group_dict = {}
            group_dict["InstanceId"] = group["InstanceId"]
            group_dict["AutoScalingGroupName"] = group["AutoScalingGroupName"]
            group_dict["region"] = region
            result.append(group_dict)
    except Exception as error:
        pass
    return result


task = Task(Workflow())
task.configure(continueOnError=True)
task.configure(inputParamsJson='''{
    "instance_ids": "instance_id",
    "region": "iter_item"
    }''')
task.configure(iterJson='''{
    "iter_enabled": true,
    "iter_list_is_const": false,
    "iter_list": "region",
    "iter_parameter": "region"
    }''')
task.configure(conditionsJson='''{
    "condition_enabled": true,
    "condition_cfg": "len(instance_id)>0",
    "condition_result": true
    }''')
task.configure(outputName="asg_instance")
task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_get_auto_scaling_instances, lego_printer=aws_get_auto_scaling_instances_printer, hdl=hdl, args=args)

<h3 id="AWS-List-All-Regions">AWS List All Regions<a class="jp-InternalAnchorLink" href="#AWS-List-All-Regions" target="_self">&para;</a></h3>
<p>In this action, we list all the available regions from AWS if the user does not provide a region as a parameter.</p>
<blockquote>
<p>Output variable: <code>region</code></p>
</blockquote>

In [4]:
#
# Copyright (c) 2021 unSkript.com
# All rights reserved.
#

from pydantic import BaseModel, Field
from typing import Dict, List
import pprint

from beartype import beartype
@beartype
def aws_list_all_regions_printer(output):
    if output is None:
        return
    pprint.pprint(output)


@beartype
def aws_list_all_regions(handle) -> List:
    """aws_list_all_regions lists all the AWS regions

        :type handle: object
        :param handle: Object returned from Task Validate

        :rtype: Result List of result
    """

    result = handle.aws_cli_command("aws ec2 describe-regions --all-regions --query 'Regions[].{Name:RegionName}' --output text")
    if result is None or result.returncode != 0:
        print("Error while executing command : {}".format(result))
        return str()
    result_op = list(result.stdout.split("\n"))
    list_region = [x for x in result_op if x != '']
    return list_region


task = Task(Workflow())
task.configure(conditionsJson='''{
    "condition_enabled": true,
    "condition_cfg": "not region",
    "condition_result": true
    }''')

task.configure(outputName="region")
task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_list_all_regions, lego_printer=aws_list_all_regions_printer, hdl=hdl, args=args)

<h3 id="Get-Unhealthy-instances-from-ASG">Get Unhealthy instances from ASG</h3>
<p>Here we will use unSkript&nbsp;<strong>Get Unhealthy instances from ASG</strong> action. This action filters all the unhealthy instances from the Auto Scaling Group. It will execute if the <code>instance_id</code> parameter is not given.</p>
<blockquote>
<p>Input parameters: <code>region</code></p>
</blockquote>
<blockquote>
<p>Output variable: <code>unhealthy_instance</code></p>
</blockquote>

In [5]:
##  Copyright (c) 2021 unSkript, Inc
##  All rights reserved.
##
from typing import Optional, Tuple
from pydantic import BaseModel, Field
from unskript.legos.utils import CheckOutput, CheckOutputStatus
from unskript.connectors.aws import aws_get_paginator
from unskript.legos.aws.aws_list_all_regions.aws_list_all_regions import aws_list_all_regions
import pprint


from beartype import beartype
@beartype
def aws_filter_unhealthy_instances_from_asg_printer(output):
    if output is None:
        return

    if isinstance(output, CheckOutput):
        print(output.json())
    else:
        pprint.pprint(output)


@beartype
def aws_filter_unhealthy_instances_from_asg(handle, region: str = "") -> CheckOutput:
    """aws_filter_unhealthy_instances_from_asg gives unhealthy instances from ASG

        :type region: string
        :param region: AWS region.

        :rtype: CheckOutput with status result and list of unhealthy instances from ASG.
    """
    result = []
    all_regions = [region]
    if not region:
        all_regions = aws_list_all_regions(handle)

    for reg in all_regions:
        try:
            asg_client = handle.client('autoscaling', region_name=reg)
            response = aws_get_paginator(asg_client, "describe_auto_scaling_instances", "AutoScalingInstances")

            # filter instances to only include those that are in an "unhealthy" state
            for instance in response:
                data_dict = {}
                if instance['HealthStatus'] == 'Unhealthy':
                    data_dict["InstanceId"] = instance["InstanceId"]
                    data_dict["AutoScalingGroupName"] = instance["AutoScalingGroupName"]
                    data_dict["region"] = reg
                    result.append(data_dict)

        except Exception as e:
            pass

    if len(result) != 0:
        return CheckOutput(status=CheckOutputStatus.FAILED,
                   objects=result,
                   error=str(""))
    else:
        return CheckOutput(status=CheckOutputStatus.SUCCESS,
                   objects=result,
                   error=str(""))






task = Task(Workflow())
task.configure(continueOnError=True)
task.configure(inputParamsJson='''{
    "region": "iter_item"
    }''')
task.configure(iterJson='''{
    "iter_enabled": true,
    "iter_list_is_const": false,
    "iter_list": "region",
    "iter_parameter": "region"
    }''')
task.configure(conditionsJson='''{
    "condition_enabled": true,
    "condition_cfg": "not instance_id",
    "condition_result": true
    }''')

task.configure(outputName="unhealthy_instance")

task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_filter_unhealthy_instances_from_asg, lego_printer=aws_filter_unhealthy_instances_from_asg_printer, hdl=hdl, args=args)

<h3 id="Modify-Output">Modify Output</h3>
<p>In this action, we modify the output from step 1 A and step 1 B to return a list of dictionary items for the unhealthy instances from ASG.</p>
<blockquote>
<p>Output variable: detach_instance_list</p>
</blockquote>

In [12]:
detach_instance_list = []
try:
    for k, v in asg_instance.items():
        for i in v:
            detach_instance_list.append(i)
except Exception as e:
    if unhealthy_instance and not asg_name:
        for k, v in unhealthy_instance.items():
            if v.status == CheckOutputStatus.FAILED:
                for instance in v.objects:
                    detach_instance_list.append(instance)
    else:
        for k, v in unhealthy_instance.items():
            if v.status == CheckOutputStatus.FAILED:
                for instance in v.objects:
                    if asg_name in instance["AutoScalingGroupName"]:
                        detach_instance_list.append(instance)

<h3 id="AWS-Detach-Instances-From-AutoScaling-Group">AWS Detach Instances From AutoScaling Group</h3>
<p>In this action, we detach the AWS unhealthy instances from the Auto Scaling Group which we get from step 1.</p>
<blockquote>
<p>Input parameters: <code>instance_ids, group_name, region</code></p>
</blockquote>
<blockquote>
<p>Output variable: <code>detach_output</code></p>
</blockquote>

In [10]:
##  Copyright (c) 2021 unSkript, Inc
##  All rights reserved.
##
from typing import List, Dict
from pydantic import BaseModel, Field
import pprint

from beartype import beartype
@beartype
def aws_detach_autoscaling_instances_printer(output):
    if output is None:
        return
    pprint.pprint(output)


@beartype
def aws_detach_autoscaling_instances(
    handle,
    instance_ids: str,
    group_name: str,
    region: str
) -> Dict:
    """aws_detach_autoscaling_instances detach instances from autoscaling group.

        :type handle: object
        :param handle: Object returned from task.validate(...).

        :type instance_ids: string
        :param instance_ids: Name of instances.

        :type group_name: string
        :param group_name: Name of AutoScaling Group.

        :type region: string
        :param region: AWS Region of autoscaling group.

        :rtype: Dict with the detach instance info.
    """

    ec2Client = handle.client("autoscaling", region_name=region)
    result = {}
    try:
        response = ec2Client.detach_instances(
            InstanceIds=[instance_ids],
            AutoScalingGroupName=group_name,
            ShouldDecrementDesiredCapacity=True
            )
        result = response
    except Exception as error:
        result["error"] = error

    return result


task = Task(Workflow())
task.configure(continueOnError=True)
task.configure(inputParamsJson='''{
    "group_name": "iter.get(\\"AutoScalingGroupName\\")",
    "instance_ids": "iter.get(\\"InstanceId\\")",
    "region": "iter.get(\\"region\\")"
    }''')
task.configure(iterJson='''{
    "iter_enabled": true,
    "iter_list_is_const": false,
    "iter_list": "detach_instance_list",
    "iter_parameter": ["instance_ids","group_name","region"]
    }''')
task.configure(conditionsJson='''{
    "condition_enabled": true,
    "condition_cfg": "len(detach_instance_list)>0",
    "condition_result": true
    }''')

task.configure(outputName="detach_output")

task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_detach_autoscaling_instances, lego_printer=aws_detach_autoscaling_instances_printer, hdl=hdl, args=args)

### Conclusion
<p>In this Runbook, we demonstrated the use of unSkript's AWS actions. This runbook helps to detach the instances from the Auto Scaling Group. To view the full platform capabilities of unSkript please visit <a href="https://us.app.unskript.io" target="_blank" rel="noopener">https://us.app.unskript.io</a></p>